### **Homework: Open-Source LLMs**

**Q1: Running Ollama with Docker**

Run ollama with Docker. What's the version of the ollama client?

*0.2.5*

**In the terminal:**
- ```
    docker run -it \
    --rm \
    -v ollama:/root/.ollama \
    -p 11434:11434 \
    --name ollama \
    ollama/ollam
    ```
- ```
    docker exec -it ollama bash
    ollama -v
    ```
- OR `docker exec -it ollama ollama -v`

**Q2: Downloading a LLM**

Download a smaller LLM - `gemma:2b`. We're interested in the model's metadata. What's the content of the file related to `gemma`?

*{"schemaVersion":2,"mediaType":"application/vnd.docker.distribution.manifest.v2+json","config":{"mediaType":"application/vnd.docker.container.image.v1+json","digest":"sha256:887433b89a901c156f7e6944442f3c9e57f3c55d6ed52042cbb7303aea994290","size":483},"layers":[{"mediaType":"application/vnd.ollama.image.model","digest":"sha256:c1864a5eb19305c40519da12cc543519e48a0697ecd30e15d5ac228644957d12","size":1678447520},{"mediaType":"application/vnd.ollama.image.license","digest":"sha256:097a36493f718248845233af1d3fefe7a303f864fae13bc31a3a9704229378ca","size":8433},{"mediaType":"application/vnd.ollama.image.template","digest":"sha256:109037bec39c0becc8221222ae23557559bc594290945a2c4221ab4f303b8871","size":136},{"mediaType":"application/vnd.ollama.image.params","digest":"sha256:22a838ceb7fb22755a3b0ae9b4eadde629d19be1f651f73efb8c6b4e2cd0eea0","size":84}]}root@93f3bb0ad1c0:~/.ollama/models/manifests/registry.ollama.ai/library/gemma#*

In [1]:
!ollama pull gemma:2b

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling c1864a5eb193...   0% ▕                ▏    0 B/1.7 GB                  pulling manifest 
pulling c1864a5eb193...   0% ▕                ▏    0 B/1.7 GB                  pulling manifest 
pulling c1864a5eb193...   0% ▕                ▏    0 B/1.7 GB                  pulling manifest 
pulling c1864a5eb193...   0% ▕                ▏    0 B/1.7 GB                  pulling manifest 
pulling c1864a5eb193...   0% ▕                ▏    0 B/1.7 GB                  pulling manifest 
pul

**In the terminal:**
- `docker exec -it ollama bash`
- `cd root/.ollama/models/manifests/registry.ollama.ai/library/gemma`
- `cat 2b`

**Q3: Running the LLM**

Test the prompt: "10 * 10". What's the answer?

*The answer is 100, which is less than 10*

**In the terminal:**
- `docker exec -it ollama ollama run gemma:2b`

**Q4: Downloading the Weights**

We don't want to have to pull the weights every time we run a docker container. Let's do it once and have them available every time we start a container. What's the size of the `ollama_files/models` folder?

*1.6G*

**In the terminal:**
- `mkdir ollama_files`
- ```
    docker run -it \
    --rm \
    -v ./ollama_files:/root/.ollama \
    -p 11434:11434 \
    --name ollama \
    ollama/ollama
    ```

In [3]:
!docker exec -it ollama ollama pull gemma:2b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling c1864a5eb193... 100% ▕████████████████▏ 1.7 GB                         
pulling 097a36493f71... 100% ▕████████████████▏ 8.4 KB                         
pulling 109037bec39c... 100% ▕████████████████▏  136 B                         
pulling 22a838ceb7fb... 100% ▕████████████████▏   84 B                         
pulling 887433b89a90... 100% ▕████████████████▏  483 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 


In [5]:
!du -sh ollama_files/models

1.6G	ollama_files/models


**Q5: Adding the Weights**

Stop the container and add the weights to a new image. First create a `Dockerfile`:
```
FROM ollama/ollama
COPY ...
```

What do you put after `COPY`?

*`ollama_files /root/.ollama`*

**Q6: Serving It**

Build and run the container. Then test it with the prompt: "What's the formula for energy?". To ensure reproducible results, set the `temperature` parameter to 0. How many completion tokens do you get in response?

*304 tokens*

In [7]:
!docker build -t ollama-gemma2b .

[+] Building 0.0s (0/1)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
[+] Building 0.1s (0/1)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile:                                            0.0s
[+] Building 0.3s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.2s
 => => transferring dockerfile: 87B                                        0.0s
 => [internal] load metadata for docker.io/ollama/ollama:latest            0.2s
[+] Building 0.5s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.2s
 => => transferring dockerfile: 87B                                        0.0s
 => [internal] load metadata for docker.

**In the terminal:**
- `docker run -it --rm -p 11434:11434 ollama-gemma2b`

In [11]:
from openai import OpenAI

prompt = "What's the formula for energy?"

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama'
)

response = client.chat.completions.create(
    model='gemma:2b',
    messages=[{"role": "user", "content": prompt}],
    temperature=0.0
)

print(response)


ChatCompletion(id='chatcmpl-309', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Sure, here's the formula for energy:\n\n**E = K + U**\n\nWhere:\n\n* **E** is the energy in joules (J)\n* **K** is the kinetic energy in joules (J)\n* **U** is the potential energy in joules (J)\n\n**Kinetic energy (K)** is the energy an object possesses when it moves or is in motion. It is calculated as half the product of an object's mass (m) and its velocity (v) squared:\n\n**K = 1/2 * m * v^2**\n\n**Potential energy (U)** is the energy an object possesses when it is in a position or has a specific configuration. It is calculated as the product of an object's mass and the gravitational constant (g) multiplied by the height or distance of the object from a reference point.\n\n**Gravitational potential energy (U)** is given by the formula:\n\n**U = mgh**\n\nWhere:\n\n* **m** is the mass of the object in kilograms (kg)\n* **g** is the acceleration due t